<a href="https://colab.research.google.com/github/SYEON9/DBArchitecture/blob/master/HW/HW2_Crawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
from bs4 import BeautifulSoup

#네이버 영화 리뷰 크롤링을 진행해보자
#서버 연결하기
URL = 'https://movie.naver.com/movie/point/af/list.naver?&page=1'
response = requests.get(URL)
print(response)
print(response.encoding)

In [ ]:
#Text 가져오기
soup = BeautifulSoup(
    response.text,
    'html.parser'
)

#print(soup)                     #제대로 나오는 것을 확인!

In [ ]:
#해당부분 가져오기
total_list = soup.find(name = 'tbody')
#print(total_list)
for title in total_list.find_all(name = 'td', attrs = {'class':'title'}):
    title = title.find('div')
    sentence = title.get_text(strip = True, separator = ",")
    #a = title[0].get_text(strip = True, separator = ",")
    #score = title.find('em').string
    print(sentence)
    #print(a)
    #print(score)
    #print(movie,",",sentence,",",score)

In [ ]:
#이게 진짜다
#지우면 안됨.

#좋아. 이제 데이터 1000개 정도를 가져와보자. 
#페이지를 100번정도 넘기면 될 것 같아!!
import time
import csv
import chardet

text = []

#크롤링하면서 바로 csv파일로 만들자.
#유니코드 변환
#sentence 비어있는거랑 신고, star없애기
with open('samples.csv','w') as fd:
    writer = csv.writer(fd,
                        delimiter = ',')
    writer.writerow(['movie','star','score','sentence','declartion'])
    
    for i in range(100):

        time.sleep(0.5)

        URL = f'https://movie.naver.com/movie/point/af/list.naver?&page={i+1}'
        response = requests.get(URL)
    
    
        #Text 가져오기
        soup = BeautifulSoup(
            response.text,
            'html.parser',
            from_encoding = 'utf-8'
        )

        #해당부분 가져오기
        total_list = soup.find(name = 'tbody')
        #print(total_list)
        print(i)
        for title in total_list.find_all(name = 'td', attrs = {'class':'title'}):
            sentence = title.get_text(strip = True, separator = "(^)")
            print(type(sentence))
            text = list(sentence.split('(^)'))
            print(text)

            #sentence.encode('euc-kr')
            writer.writerow(text)
            #print(sentence)
            
    
    



In [ ]:
#만들어진 csv파일을 불러와서 전처리를 진행해보자.

#데이터 불러오기
import pandas as pd
csv_test = pd.read_csv('samples.csv')

In [ ]:
csv_test.shape                           #데이터 확인

In [ ]:
csv_test.head()

In [ ]:
#신고되어 삭제된 한줄평이나 한줄평이 없는 경우는 제외하자. 
edit_csv_test = csv_test
edit_csv_test.info()           #데이터 정보 확인. 

In [ ]:
edit_csv_test.isnull().sum()    #declartion에 72개의 None값이 존재한다. 

In [ ]:
#결측값이 있다는 것은 sentence값이 없어 "신고"가 sentence자리에 들어갔다는 의미이므로 해당 열은 삭제한다.
edit_csv_test = edit_csv_test.dropna()       #결측치 제거

In [ ]:
edit_csv_test.shape                          #결측치 제거 확인

In [ ]:
#이제 필요없는 열을 제거해보자. 
edit_csv_test = edit_csv_test.drop(['star','declartion'], axis = 1)

In [ ]:
edit_csv_test.shape                          #제거 확인

In [ ]:
edit_csv_test

In [ ]:
#데이터셋 완성!!
#이제 csv파일로 다시 저장하자!
edit_csv_test.to_csv("edit_samples.csv", encoding = 'EUC-KR')